In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm.notebook import tqdm
import re

In [2]:
df.head()

NameError: name 'df' is not defined

In [3]:
df['code2'][0]

NameError: name 'df' is not defined

In [8]:
df = pd.read_csv('./test.csv')

In [9]:
def del_comment(text):
    in_comment = False
    i = 0
    while i < len(code):
        # 블록 주석 시작
        if text[i:i+2] == '/*':
            in_comment = True
            start = i
        # 블록 주석 끝
        if in_comment and text[i:i+2] == '*/':
            in_comment = False
            end = i + 2
            text = text[:start] + text[end:]
            i = start - 1  # 제거 후 인덱스 조정
        i += 1
    
    # 한 줄 주석 제거
    lines = text.split('\n')
    new_lines = []
    for line in lines:
        # '//'가 있는지 확인하고, 있다면 해당 위치 이전까지만 포함
        comment_start = line.find('//')
        if comment_start != -1:
            line = line[:comment_start]
        if line.strip():  # 공백 라인 제거
            new_lines.append(line)
    
    text = ' '.join(new_lines)
    return text

In [10]:
def del_function(text):
    return re.sub('\s', ' ',re.sub('\t',' ',re.sub('\n',' ',text)))

def only_ascii(text):
    return text.encode('ascii', 'ignore').decode()

df['code1'] = df['code1'].apply(del_comment)
df['code2'] = df['code2'].apply(del_comment)
df = df.applymap(del_function)
df['code1'] = df['code1'].apply(only_ascii)
df['code2'] = df['code2'].apply(only_ascii)

KeyboardInterrupt: 

In [87]:
df.to_csv('test_noascii.csv')

In [4]:
import os
import glob
import re

def read_cpp_codes_from_folders(root_dir):
    """
    지정된 루트 디렉토리 아래의 모든 폴더에서 .cpp 파일들을 읽어서
    {라벨: [파일1의 내용, 파일2의 내용, ...]} 형태의 딕셔너리로 반환합니다.
    폴더 이름에서 숫자 부분을 라벨로 사용합니다.
    """
    label_to_code = {}
    
    # 각 라벨(폴더)를 순회합니다.
    for folder in tqdm(os.listdir(root_dir)):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            # 폴더 이름에서 숫자 부분(라벨)을 추출합니다.
            label_match = re.search(r'\d+', folder)
            if label_match:
                label = int(label_match.group(0))
                cpp_files = glob.glob(os.path.join(folder_path, '*.cpp'))
                code_list = []
                
                # 각 .cpp 파일을 열고 내용을 읽습니다.
                for cpp_file in cpp_files:
                    with open(cpp_file, 'r', encoding='utf-8') as file:
                        code = file.read()
                            
                            
                        in_comment = False
                        i = 0
                        while i < len(code):
                            # 블록 주석 시작
                            if code[i:i+2] == '/*':
                                in_comment = True
                                start = i
                            # 블록 주석 끝
                            if in_comment and code[i:i+2] == '*/':
                                in_comment = False
                                end = i + 2
                                code = code[:start] + code[end:]
                                i = start - 1  # 제거 후 인덱스 조정
                            i += 1

                        # 한 줄 주석 제거
                        lines = code.split('\n')
                        new_lines = []
                        for line in lines:
                            # '//'가 있는지 확인하고, 있다면 해당 위치 이전까지만 포함
                            comment_start = line.find('//')
                            if comment_start != -1:
                                line = line[:comment_start]
                            if line.strip():  # 공백 라인 제거
                                new_lines.append(line)

                        new_content = ' '.join(new_lines)

                        code = re.sub('\n',' ',code)
                        code = re.sub('\t',' ',code)
                        code = re.sub('\s',' ',code)
                        #code = code.encode('ascii', 'ignore').decode()
                        #code = code.lower()
                        code_list.append(code)
                
                # 추출한 라벨을 키로 하여 코드 리스트를 저장합니다.
                if label not in label_to_code:
                    label_to_code[label] = code_list
                else:
                    label_to_code[label].extend(code_list)
            
    return label_to_code

# 루트 디렉토리 경로를 설정하세요.
root_dir = './train_code'
cpp_codes = read_cpp_codes_from_folders(root_dir)

# 결과를 확인합니다.
for label, codes in cpp_codes.items():
    print(f"Label: {label}, Number of CPP files: {len(codes)}")


  0%|          | 0/500 [00:00<?, ?it/s]

Label: 156, Number of CPP files: 500
Label: 317, Number of CPP files: 500
Label: 61, Number of CPP files: 500
Label: 390, Number of CPP files: 500
Label: 171, Number of CPP files: 500
Label: 454, Number of CPP files: 500
Label: 199, Number of CPP files: 500
Label: 200, Number of CPP files: 500
Label: 195, Number of CPP files: 500
Label: 361, Number of CPP files: 500
Label: 452, Number of CPP files: 500
Label: 403, Number of CPP files: 500
Label: 388, Number of CPP files: 500
Label: 301, Number of CPP files: 500
Label: 99, Number of CPP files: 500
Label: 98, Number of CPP files: 500
Label: 230, Number of CPP files: 500
Label: 492, Number of CPP files: 500
Label: 401, Number of CPP files: 500
Label: 269, Number of CPP files: 500
Label: 475, Number of CPP files: 500
Label: 194, Number of CPP files: 500
Label: 192, Number of CPP files: 500
Label: 351, Number of CPP files: 500
Label: 486, Number of CPP files: 500
Label: 107, Number of CPP files: 500
Label: 467, Number of CPP files: 500
Labe

In [5]:
import pandas as pd


# 정수형으로 키 변환
cpp_codes_dict_int_keys = {int(key): value for key, value in cpp_codes.items()}

# DataFrame 생성을 위한 데이터 준비
data = []
for label, codes in cpp_codes_dict_int_keys.items():
    for code in codes:
        data.append({'label': label, 'code': code})

# DataFrame 생성
df = pd.DataFrame(data)
df = df.sort_values(by=['label'], axis=0)
df['label'] = df['label'].apply(lambda x: x -1)
df.head()  # DataFrame의 처음 몇 줄을 출력하여 확인


,label,code
50408,0,#include<iostream> #include<cstdlib> #include<...
50409,0,#define _USE_MATH_DEFINES #include <iostream>...
50474,0,#include <iostream> using namespace std; usin...
50475,0,#include <stdio.h> #include <iostream> int ma...
50476,0,"#include<bits/stdc++.h> #define rep(i,n) for (..."


In [6]:
df.to_csv('final_train.csv',index=False)

In [7]:
print(df['code'][0])

#include <iostream> #include <stdio.h> #include <math.h> using namespace std; int main(){  double a,b,c,d,s,l,h,PI=3.14159265358979;  cin>>a>>b>>c;  c=(c*PI)/180;  s=(0.5*a*b)*sin(c);  d=sqrt(((a*a)+(b*b))-(2*a*b*cos(c)));  h=(s*2)/a;  l=a+b+d;  printf("%.8lf\n%.8lf\n%.8lf\n",s,l,h);  return 0; }


In [176]:
df1 = pd.read_csv('submission_plz.csv')
df2 = pd.read_csv('submission_ja.csv')
df3 = pd.read_csv('submission_good.csv')
df4 = pd.read_csv('submission_unix.csv')

In [177]:
df1.head()

,pair_id,similar
0,TEST_000000,1
1,TEST_000001,0
2,TEST_000002,1
3,TEST_000003,0
4,TEST_000004,1


In [178]:
tmp = np.where(df1['similar']+df2['similar']+df3['similar']+df4['similar']>2,1,0)

In [179]:
df2['similar'] = tmp

In [180]:
df2.to_csv('submission_vote.csv',index=False)

In [181]:
df2['similar']

0         1
1         0
2         1
3         0
4         1
         ..
594995    0
594996    0
594997    1
594998    1
594999    0
Name: similar, Length: 595000, dtype: int64

In [182]:
sum(tmp)

292156

In [183]:
df2.head()

,pair_id,similar
0,TEST_000000,1
1,TEST_000001,0
2,TEST_000002,1
3,TEST_000003,0
4,TEST_000004,1
